# MCQ (Multiple Choice Question) 평가 튜토리얼

## MCQDataset

이 튜토리얼에서는 Huggingface의 객관식 dataset을 불러와서 평가 후 재업로드하는 과정까지 경험해볼 것입니다.

### 1. 데이터셋 불러오기
먼저 HuggingFace Hub에서 데이터셋을 불러오는 방법을 알아보겠습니다:

In [13]:
from langmetrics.llmdataset import LLMDataset
from langmetrics.llmtestcase import LLMTestCase
from datasets import load_dataset
import pandas as pd
from dotenv import load_dotenv

In [14]:
load_dotenv(override=True)

True

In [ ]:
dataset = load_dataset('sickgpt/001_MedQA_raw')

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'expected_output', 'choices'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'expected_output', 'choices'],
        num_rows: 1273
    })
})

이제 LLMDataset을 이용해서 불러와봅시다.

먼저 LLMTestCase는 input, choices, expected_output을 고정으로 받습니다. 그런데 위에 Dataset은 input이 question이라는 열로 되어있네요. field_mapping 인자를 이용해서 column을 매핑해주겠습니다.

In [ ]:
LLMTestCase.__annotations__

{'input': str,
 'output': typing.Optional[str],
 'expected_output': str,
 'context': typing.Optional[typing.List[str]],
 'retrieval_context': typing.Optional[typing.List[str]],
 'choices': typing.Optional[str]}

In [ ]:
# 예시 사용법
field_mapping = {
    'input': 'question',  # 데이터셋의 'question' 필드를 'input'으로 매핑
    'expected_output': 'expected_output',
    'choices': 'choices'
}

In [ ]:
dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping)

In [ ]:
print(len(dataset))

10178


In [ ]:
test_dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping, split='test')

In [10]:
len(test_dataset)

1273

In [11]:
dataset

LLMDataset(Pandas DataFrame with 10178 rows)

이제 evaluate을 진행해봅시다.

In [12]:
from langmetrics.llmfactory import LLMFactory
from langmetrics.config import ModelConfig

ModuleNotFoundError: No module named 'ahocorasick'

In [ ]:
LLMFactory.get_model_list()

['gpt-4o',
 'gpt-4o-mini',
 'deepseek-v3',
 'deepseek-reasoner',
 'claude-3.7-sonnet',
 'claude-3.5-sonnet',
 'claude-3.5-haiku',
 'naver',
 'gemini-2.0-flash']

In [ ]:
# # 커스텀 모델 설정 생성
# custom_config = ModelConfig(
#     model_name="Qwen/Qwen2.5-3B-Instruct",
#     api_base="http://qwen3b:8000/v1",
#     api_key='EMPTY',
#     max_tokens=32000,
#     seed=66,
#     provider="openai"
# )

In [ ]:
# # localllm은 서버를 local에서 실행시키기 때문에 부팅되는 시간이 존재합니다.
# custom_llm = LLMFactory.create_llm(custom_config, temperature=1.0)

In [16]:
from langmetrics.llmfactory import LLMFactory
# LLM 모델 생성
deepseek_llm= LLMFactory.create_llm('gpt-4o-mini')

In [17]:
from langmetrics.metrics import MCQMetric
metric = MCQMetric(
    output_model=deepseek_llm,
    template_language='ko',  # 'ko' 또는 'en'
    output_template_type='reasoning'  # 'reasoning' 또는 'only_answer'
)

async를 통해서 빠르게 추론을 할 것입니다.

In [18]:
import nest_asyncio
nest_asyncio.apply()

In [24]:
results = await metric.ameasure(dataset[:500])

100%|██████████| 500/500 [00:12<00:00, 40.08it/s] 


In [25]:
results

LLMDataset(Pandas DataFrame with 500 rows)

In [21]:
# r1_results = await r1_metric.ameasure(test_dataset[:10])

약 1200개의 달하는 test를 단 30초만에 모두 추론한 것을 확인할 수 있습니다!

In [ ]:
dataset = LLMDataset.from_huggingface_hub('sickgpt/001_MedQA_raw', field_mapping=field_mapping)

In [26]:
results.df

,input,output,scoring_model_output,expected_output,context,retrieval_context,choices,score,metadata
0,A 23-year-old pregnant woman at 22 weeks gesta...,"{\n ""reasoning"": ""이 경우, 환자는 22주차의 임신 중이며, 배뇨 ...",None,D,None,None,"[Ampicillin, Ceftriaxone, Doxycycline, Nitrofu...",1,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
1,A 3-month-old baby died suddenly at night whil...,"{\n ""reasoning"": ""이 문제는 아기의 갑작스러운 사망 원인과 예방 조...",None,A,None,None,[Placing the infant in a supine position on a ...,1,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
2,A mother brings her 3-week-old infant to the p...,"{\n ""reasoning"": ""이 아기는 생후 3주이며, 최근 4일간 수유 후 ...",None,A,None,None,"[Abnormal migration of ventral pancreatic bud,...",0,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
3,A pulmonary autopsy specimen from a 58-year-ol...,"{\n ""reasoning"": ""환자는 수술 후 회복 중 갑작스러운 호흡 곤란과 ...",None,A,None,None,"[Thromboembolism, Pulmonary ischemia, Pulmonar...",1,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
4,A 20-year-old woman presents with menorrhagia ...,"{\n ""reasoning"": ""환자는 생리량이 많고, 멍이 잘 드는 증상을 보이...",None,D,None,None,"[Hemophilia A, Lupus anticoagulant, Protein C ...",1,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
...,...,...,...,...,...,...,...,...,...
495,A 54-year-old man is brought to the physician ...,"{\n ""reasoning"": ""환자는 54세 남성으로, 걷는 데 점점 어려움을 ...",None,A,None,None,"[Multiple system atrophy, Friedreich ataxia, C...",0,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
496,A 28-year-old primigravid woman at 36 weeks' g...,"{\n ""reasoning"": ""환자는 36주 임신 상태에서 규칙적인 자궁 수축을...",None,C,None,None,"[Offer local or regional anesthesia, Admit for...",1,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
497,A 28-year-old woman is brought to the emergenc...,"{\n ""reasoning"": ""환자의 증상은 어린 시절부터 지속된 실신과 함께 ...",None,C,None,None,"[Calcium gluconate, Flecainide, Magnesium sulf...",0,"{'output_model_name': 'gpt-4o-mini', 'token_us..."
498,A 61-year-old G4P3 presents with a 5-year hist...,"{\n ""reasoning"": ""환자는 기침, 재채기, 신체적 노력 시 무의식적으...",None,D,None,None,"[Normal residual volume, involuntary detrusor ...",1,"{'output_model_name': 'gpt-4o-mini', 'token_us..."


In [25]:
print([i.score for i in results])

[0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 

In [26]:
scores = sum([i.score for i in results]) / len(results)

In [30]:
print(scores)

0.47540983606557374


In [28]:
print(results[0])

LLMResult(input='A 35-year-old woman presents to her primary care provider concerned that she may be pregnant. She has a history of regular menstruation every 4 weeks that lasts about 4 days with mild to moderate bleeding, but she missed her last period 2 weeks ago. A home pregnancy test was positive. She has a 6-year history of hyperthyroidism that is well-controlled with daily methimazole. She is currently asymptomatic and has no complaints or concerns. A blood specimen is taken and confirms the diagnosis. Additionally, her thyroid-stimulating hormone (TSH) is 2.0 μU/mL. Which of the following is the next best step in the management of this patient?', student_answer='{\n    "reasoning": "<Pregnant patients with hyperthyroidism are typically managed by discontinuing methimazole as it can cross the placenta and potentially harm the fetus. Given that this patient\'s TSH is 2.0 μU/mL, which is mildly elevated, they are still within a range that suggests hyperthyroidism but is not dangero

In [29]:
results.df['metadata']

0      {'student_template_language': 'en', 'student_m...
1      {'student_template_language': 'en', 'student_m...
2      {'student_template_language': 'en', 'student_m...
3      {'student_template_language': 'en', 'student_m...
4      {'student_template_language': 'en', 'student_m...
                             ...                        
483    {'student_template_language': 'en', 'student_m...
484    {'student_template_language': 'en', 'student_m...
485    {'student_template_language': 'en', 'student_m...
486    {'student_template_language': 'en', 'student_m...
487    {'student_template_language': 'en', 'student_m...
Name: metadata, Length: 488, dtype: object

In [27]:
dataset = load_dataset('sickgpt/001_MedQA_processed', split='train')

In [30]:
dataset[]

Dataset({
    features: ['question', 'expected_output', 'choices', 'question_ko', 'voca', 'answer', 'situation', 'tone', 'realqa_ko'],
    num_rows: 10180
})

In [31]:
processed_df = dataset.to_pandas()

In [34]:
processed_df[:500][['question_ko']]

,question_ko
0,23세 임신 22주 여성이 배뇨 시 작열감을 호소하며 내원했다. 증상은 하루 전 시...
1,3개월 된 아이가 밤에 잠든 상태에서 갑자기 사망했다. 어머니는 아침에 깨어난 후에...
2,3주 된 영아를 데리고 온 어머니는 아이의 수유 습관이 걱정되어 소아청소년과 의사를...
3,급성 저산소성 호흡부전으로 사망한 58세 여성의 폐 부검 표본을 검사했다. 3개월 ...
4,"20세 여성이 지난 몇 년 동안 월경과다를 주소로 내원했다. ""항상 생리량이 많았다..."
...,...
495,54세 남성이 지난 3개월 동안 걷기가 점점 어려워져 아내와 함께 병원을 찾았다. ...
496,36주 임신 중인 28세 초임 여성이 2시간 동안 지속된 자궁 수축으로 응급실을 방...
497,28세 여성이 직장에서 실신한 후 머리를 부딪혀 친구와 함께 응급실로 이송되었다. ...
498,"61세 여성으로 산과력 4-3-?-?이며, 기침, 재채기, 신체 활동 시 비자발적인..."


In [47]:
template = metric.template_for_answer

In [48]:
print(template)

input_variables=['choices', 'question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 유능한 전문가입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['choices', 'question'], input_types={}, partial_variables={}, template='다음의 객관식 문제를 풀어주세요.\n추론 과정을 설명하고 정답은 알파벳(A, B, C, D 등)으로만 답해주세요.\n\n**\n중요 : 반드시 JSON 형식으로만 답변해주세요. \'reasoning\' 키에는 추론 과정을 한국어로 작성합니다.\nJSON 예시:\n{{\n"reasoning": "<추론_과정. 차근차근 생각하세요.>. 따라서 답은 <정답>입니다",\n"answer": "<정답>"\n}}\n**\n\n문제:\n{question}\n\n보기:\n{choices}\n\nJSON:\n'), additional_kwargs={})]


In [ ]:
template.format_promt()

In [35]:
results = results.df

In [52]:
print(template)

input_variables=['choices', 'question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 유능한 전문가입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['choices', 'question'], input_types={}, partial_variables={}, template='다음의 객관식 문제를 풀어주세요.\n추론 과정을 설명하고 정답은 알파벳(A, B, C, D 등)으로만 답해주세요.\n\n**\n중요 : 반드시 JSON 형식으로만 답변해주세요. \'reasoning\' 키에는 추론 과정을 한국어로 작성합니다.\nJSON 예시:\n{{\n"reasoning": "<추론_과정. 차근차근 생각하세요.>. 따라서 답은 <정답>입니다",\n"answer": "<정답>"\n}}\n**\n\n문제:\n{question}\n\n보기:\n{choices}\n\nJSON:\n'), additional_kwargs={})]


In [ ]:
template.format_prompt('')

In [54]:
final_dfs.iloc[0]['question_ko']

'23세 임신 22주 여성이 배뇨 시 작열감을 호소하며 내원했다. 증상은 하루 전 시작되었고, 물을 더 많이 마시고 크랜베리 추출물을 복용했음에도 악화되었다고 한다. 그 외에는 전반적으로 기분이 좋으며, 임신 관리를 위해 의사와 정기적으로 상담 중이다. 체온은 36.5°C, 혈압은 122/77 mmHg, 맥박은 80회/분, 호흡은 19회/분, 산소포화도는 98%이다. 신체 검사에서 갈비척추각 압통은 없었고, 임신자궁이 있었다. 이 환자에게 가장 적절한 치료는?'

In [56]:
final_dfs.iloc[0]['choices']

['Ampicillin', 'Ceftriaxone', 'Doxycycline', 'Nitrofurantoin']

In [58]:
choices_str = '\n'.join(f"{chr(65 + i)}: {value}" for i, value in enumerate(final_dfs.iloc[0]['choices']))
template.format_messages(question=final_dfs.iloc[0]['question_ko'], choices=choices_str)[1].content

'다음의 객관식 문제를 풀어주세요.\n추론 과정을 설명하고 정답은 알파벳(A, B, C, D 등)으로만 답해주세요.\n\n**\n중요 : 반드시 JSON 형식으로만 답변해주세요. \'reasoning\' 키에는 추론 과정을 한국어로 작성합니다.\nJSON 예시:\n{\n"reasoning": "<추론_과정. 차근차근 생각하세요.>. 따라서 답은 <정답>입니다",\n"answer": "<정답>"\n}\n**\n\n문제:\n23세 임신 22주 여성이 배뇨 시 작열감을 호소하며 내원했다. 증상은 하루 전 시작되었고, 물을 더 많이 마시고 크랜베리 추출물을 복용했음에도 악화되었다고 한다. 그 외에는 전반적으로 기분이 좋으며, 임신 관리를 위해 의사와 정기적으로 상담 중이다. 체온은 36.5°C, 혈압은 122/77 mmHg, 맥박은 80회/분, 호흡은 19회/분, 산소포화도는 98%이다. 신체 검사에서 갈비척추각 압통은 없었고, 임신자궁이 있었다. 이 환자에게 가장 적절한 치료는?\n\n보기:\nA: Ampicillin\nB: Ceftriaxone\nC: Doxycycline\nD: Nitrofurantoin\n\nJSON:\n'

In [38]:
results['question_ko'] = processed_df[:500][['question_ko']]

In [77]:
final_dfs = results[results['score'] == 1][['question_ko', 'output', 'choices']]

In [78]:
def formatting_question(question_ko, choices):
    choices_str = '\n'.join(f"{chr(65 + i)}: {value}" for i, value in enumerate(choices))
    return template.format_messages(question=question_ko, choices=choices_str)[1].content

In [79]:
final_dfs['formatting_question'] = final_dfs.apply(lambda row: formatting_question(row['question_ko'], row['choices']), axis=1)

In [80]:
def process(question_ko, output):
    return [{"role": "system", "content" : question_ko}, {"role": "assistant", "content" : output}]

In [81]:
final_dfs['dialogue'] = final_dfs.apply(lambda row: process(row['formatting_question'], row['output']), axis=1)

In [82]:
final_dfs = final_dfs[['question_ko', 'dialogue']]

In [85]:
final_dfs.to_json("medqa_4o_mini.json", orient="records", force_ascii=False, lines=True)